In [ ]:
from app.backend.core.file_processor import FileProcessor
from app.backend.core.analyzer import analyze_candidate
resume = await FileProcessor().extract_from_file("data/ZacharyBiery_CV_2024.docx")
print(analyze_candidate(resume))

2025-07-07 00:02:23 | app.backend.core.file_processor | INFO | Extracting text from local file: data/ZacharyBiery_CV_2024.docx
2025-07-07 00:02:23 | app.backend.core.file_processor | INFO | Using .DOCX parser for file: data/ZacharyBiery_CV_2024.docx
2025-07-07 00:02:23 | app.backend.core.file_processor | INFO | Extracting text from 'data/ZacharyBiery_CV_2024.docx' using local Docx parser (python-docx)
2025-07-07 00:02:23 | app.backend.services.groq | DEBUG | Retrieving Groq API key from Azure Key Vault...
2025-07-07 00:02:26 | app.backend.services.groq | DEBUG | Groq API key retrieved.
2025-07-07 00:02:26 | app.backend.services.groq | INFO | Running structured function-calling query...


c:\Users\Zbier\source\repos\resume-screening\app\backend\services\groq.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  completion = self.llm.predict_messages(


2025-07-07 00:02:29 | app.backend.services.groq | DEBUG | Raw function call arguments: {"candidate_name":"Zachary Biery","certificate":[],"comment":"Zachary Biery is a highly skilled data science professional with experience in machine learning, data analytics, and software development. He has a strong educational background with a GPA of 4.0 and is expected to complete his Master's in Business Analytics in 2025.","education":[{"field":"Business Analytics","gpa":null,"institution":"University of Cincinnati","level":"Master's","year":"2025"},{"field":"Business Analytics","gpa":"4.0","institution":"University of Cincinnati","level":"Bachelor's","year":"2024"}],"email":"bieryzt@mail.uc.edu","job_recommended":["Data Engineer","Business Intelligence Analyst"],"phone_number":"(513) 491-2255","responsibilities":["Train and implement NLP machine learning models","Create multi-class classification models","Develop full-stack software","Automate internal workflows","Develop Power BI reports"],"r

In [1]:
from app.backend.services.factory import AIServiceFactory

service = await AIServiceFactory.create_service()
response = await service.query("What is the area of the UK?")
print(response)

2025-07-07 00:38:58 | app.backend.services.groq | DEBUG | Retrieving Groq API key from Azure Key Vault...
2025-07-07 00:39:01 | app.backend.services.groq | DEBUG | Groq API key retrieved.


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000255A5E7B1C0>


2025-07-07 00:39:01 | app.backend.services.groq | INFO | Calling Groq model 'llama3-70b-8192' using ainvoke...
The total area of the United Kingdom (UK) is approximately 243,610 square kilometers (93,625 square miles). This includes:

* England: 130,279 km² (50,261 sq mi)
* Scotland: 78,772 km² (30,414 sq mi)
* Wales: 20,779 km² (8,023 sq mi)
* Northern Ireland: 14,160 km² (5,470 sq mi)

Note: These figures are approximate and may vary slightly depending on the source.


In [1]:
from app.backend.common.config import AppConfig
from app.backend.services.factory import AIServiceFactory
from app.backend.services.schema import AIServiceInterface

# Define your system prompt and function schema
from app.backend.llm.prompts import system_prompt_candidate 
from app.backend.llm.functions import fn_candidate_analysis

from app.backend.core.file_processor import FileProcessor
resume = await FileProcessor().extract_from_file("data/ZacharyBiery_CV_2024.pdf")

# Main analysis function
def analyse_candidate(candidate_data: str) -> dict:
    # Create service using factory (Groq or Azure OpenAI based on config)
    ai_service: AIServiceInterface = AIServiceFactory.create_service()

    # Run structured query
    return ai_service.structured_query(
        text=candidate_data,
        system_prompt=system_prompt_candidate,
        functions=fn_candidate_analysis
    )

print(analyse_candidate(resume))

2025-07-06 23:10:22 | app.backend.core.file_processor | INFO | Extracting text from local file: data/ZacharyBiery_CV_2024.pdf
2025-07-06 23:10:22 | app.backend.core.file_processor | INFO | Using .PDF parser for file: data/ZacharyBiery_CV_2024.pdf
2025-07-06 23:10:22 | app.backend.core.file_processor | INFO | Extracting text from 'data/ZacharyBiery_CV_2024.pdf' using local PDF parser (pypdf)
2025-07-06 23:10:26 | app.backend.services.groq | INFO | Running structured function-calling query...


c:\Users\Zbier\source\repos\resume-screening\app\backend\services\groq.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  completion = self.llm.predict_messages(


{'candidate_name': 'Zachary Biery', 'certificate': [], 'comment': '', 'education': [{'degree': 'M.S. Business Analytics', 'institution': 'University of Cincinnati', 'year': 'Expected: April 2025'}, {'degree': 'B.S. Business Analytics', 'gpa': '4.0', 'institution': 'University of Cincinnati', 'year': 'April 2024'}], 'email': 'bieryzt@mail.uc.edu', 'experience': ['Data Science Intern, Commercial Operations – Analytics & Business Intelligence Team at Medpace, Cincinnati OH (May 2024 – Present)', 'Systems & Business Intelligence Intern, Production Control – Systems Team at Toyota North America, Georgetown KY (January 2023 – April 2023)', 'Data Analyst Intern, Enterprise Sourcing – Supplier Solutions Team at Kroger, Cincinnati OH (May 2022 – August 2022)'], 'job_recommended': [], 'phone_number': '(513) 491-2255', 'responsibility': ['Trained & implemented NLP machine learning models using Scikit-Learn & TensorFlow', 'Created multi-class classification models to predict patient biopsy eligibi

In [13]:
from datetime import date

prompt = f"""
You are an expert HR recruiter and resume screener. Your task is to extract relevant information from a resume and organize it into structured categories using the format below.

---

**Categories**:

1. **Synopsis** - A personal summary, description, or “about me” section.  
2. **Experience** - Includes jobs/internships under `<job>` and standalone projects under `<project>`.  
3. **Education** - Each degree appears under a `<degreeEntry>` with optional GPA and graduation year.  
4. **Skills** - Divided into technical and non-technical.

---

**Important Instructions**:
- If no synopsis is present, insert the word `Unknown` in the `<synopsis>` tag.
- Do **not** invent or infer missing information. Only extract what is explicitly present.
- Use the exact XML structure and tag names provided.
- All durations must use time units (e.g., `"3 months"`, `"2 years"`).
- Only include UTF-8 characters. Strip bullets, markdown, or decorative formatting.
    - If only a single month is present in the date range, the duration is `"1 month"`
    - For reference, `"Present"` in a job duration refers to {date.today()}

---

**Format your output as follows**:

```xml
<resume>
    <synopsis>
        INSERT SYNOPSIS INFORMATION HERE
    </synopsis>
    <experience>
        <job>
            <company>
                INSERT COMPANY NAME
            </company>
            <duration>
                INSERT JOB DURATION
            </duration>
            <description>
                INSERT JOB DESCRIPTION
            </description>
        </job>
        <!-- Repeat <job> as needed -->

        <project>
            <name>
                INSERT PROJECT NAME
            </name>
            <duration>
                INSERT PROJECT DURATION (if available)
            </duration>
            <description>
                INSERT PROJECT DESCRIPTION
            </description>
        </project>
        <!-- Repeat <project> as needed -->
    </experience>
    <education>
        <degreeEntry>
            <university>
                INSERT UNIVERSITY NAME
            </university>
            <degree>
                <level>
                    INSERT DEGREE LEVEL (e.g., Doctorate, Masters, Baccalaureate)
                </level>
                <name>
                    INSERT DEGREE NAME (e.g., Computer Science)
                </name>
            </degree>
            <graduationYear>
                INSERT GRADUATION YEAR (if available)
            </graduationYear>
            <gpa>
                INSERT GPA (if available)
            </gpa>
        </degreeEntry>
        <!-- Repeat <degreeEntry> as needed -->
    </education>
    <skills>
        <technical>
            <software>
                LIST SOFTWARE SKILLS
            </software>
            <languages>
                LIST PROGRAMMING LANGUAGES
            </languages>
            <frameworks>
                LIST FRAMEWORKS
            </frameworks>
            <tools>
                LIST TECHNICAL TOOLS
            </tools>
        </technical>
        <nontechnical>
            LIST NON-TECHNICAL SKILLS
        </nontechnical>
    </skills>
</resume>

---

You will now be given a resume in plain text. Extract and organize the information as described above.
**Resume**:  
{resume}
"""

In [5]:
from app.backend.common.config import AppConfig
from app.backend.services.factory import AIServiceFactory
from app.backend.services.schema import AIServiceInterface

# Define your system prompt and function schema
from app.backend.llm.prompts import system_prompt_job  
from app.backend.llm.functions import fn_job_analysis     

# Main analysis function
def analyse_job(job_data: str) -> dict:
    # Create service using factory (Groq or Azure OpenAI based on config)
    ai_service: AIServiceInterface = AIServiceFactory.create_service()

    # Run structured query
    return ai_service.structured_query(
        text=job_data,
        system_prompt=system_prompt_job,
        functions=fn_job_analysis
    )


print(analyse_job(job_data))


2025-07-06 22:26:33 | app.backend.services.groq | INFO | Running structured function-calling query...
{'certificate': ['AWS', 'TensorFlow'], 'degree': ["Bachelor's degree in Computer Science or a related field"], 'experience': ['machine learning engineer with experience in Python, PyTorch, and MLOps'], 'responsibility': ['model deployment', 'data pipeline design', 'working in a fast-paced, collaborative environment'], 'soft_skill': ['collaboration'], 'technical_skill': ['Python', 'PyTorch', 'MLOps']}


In [20]:
import json
import time
import jsbeautifier

from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from langchain.schema import HumanMessage, SystemMessage
from langchain_groq import ChatGroq

from app.backend.common.config import AppConfig

# Load app config and secret
config = AppConfig()
credential = DefaultAzureCredential()
secret_client = SecretClient(vault_url=config.keyvault_url, credential=credential)
groq_secret = secret_client.get_secret(config.groq_secret_name)

# Utility to parse function_call output into JSON
def output2json(output: dict) -> dict:
    """Convert LLM function_call argument string to JSON object."""
    opts = jsbeautifier.default_options()
    raw_args = output.get("function_call", {}).get("arguments", "")
    return json.loads(jsbeautifier.beautify(raw_args, opts))

# Main analysis function
def analyse_job(job_data: str) -> dict:
    start = time.time()

    # Initialize Groq Chat LLM
    llm = ChatGroq(
        api_key=groq_secret.value,
        model=config.groq_model,
        temperature=0.5
    )

    # Send message with system and user prompt
    completion = llm.predict_messages(
        messages=[
            SystemMessage(content=system_prompt_job),
            HumanMessage(content=job_data)
        ],
        functions=fn_job_analysis,
        function_call="auto"
    )

    # Parse function call output
    output_analysis = completion.additional_kwargs
    json_output = output2json(output_analysis)

    return json_output


print(analyse_job(job_data))


{'certificate': [], 'degree': ["Bachelor's degree or higher in Artificial Intelligence, Computer or Data Science, or related field. Master's degree preferred"], 'experience': ['Implementing and fine-tuning Large Language Models or interactive AI applications (>1 year post-graduate experience)'], 'responsibility': ['Design, implement, and deploy machine learning models and systems to solve complex problems and drive business outcomes;', 'Research, develop, and implement machine learning algorithms and models for tasks such as classification, regression, clustering, anomaly detection, and recommendation systems;', 'Research, develop, and implement AI tools such as NLP, LLM, and IA;', 'Collect, preprocess, and curate large datasets required for training generative models;', 'Experiment with different machine learning techniques and algorithms, including supervised, unsupervised, semi-supervised, reinforcement, and deep learning;', 'Design and optimize machine learning pipelines and workfl